## EDA
#### 1. Target variable distribution
#### 2. Feature visualization with Target variable
#### 3. Imputing missing values
#### 4. Fix Skewed features
#### 5. Feature Creation
#### 6. Feature Transformation
#### 7. Training a model
#### 8. Prediction

In [ ]:
##pip install lightgbm
#conda install -c conda-forge xgboost
#pip install xgboost

In [1]:
#importing necessary models and libraries

#Math tools
from scipy import stats
from scipy.stats import skew,norm  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import scipy.stats as stats


#Visualizing tools
import seaborn as sns
import matplotlib.pyplot as plt

#preprocessing tools
import pandas as pd
import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

#ML Algo
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
import sklearn.linear_model as linear_model
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor

In [2]:
df =  pd.read_csv('C://Data/usa.csv')

In [3]:
df.head()

,Id,AboutMe,AboutMe_length,CreationDate,LastAccessDate,activity_in_months,DisplayName,Location,UpVotes,DownVotes,...,ph_PostUnmigrated,ph_UnknownModeratorEvent,ph_UnknownEvent,ph_CommentDiscussionMovedToChat,ph_PostNoticeRemoved,ph_PostNoticeAdded,ph_PostMergeSource,ph_PostMigratedAway,ph_PostMergeDestination,ph_PostMigratedHere
0,7784035,<p>This product may also help you to feel full...,1407.0,2017-03-29 06:30:11.463 UTC,2017-03-29 06:30:11.463 UTC,0.0,berwenally,New York,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9126518,<p>A simple way to keep your hands moisturized...,622.0,2017-12-21 10:07:00.237 UTC,2017-12-21 10:07:00.237 UTC,0.0,TeresaEWilliams,United States,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10021482,<p>Completely covers and protects your body pi...,370.0,2018-07-02 12:09:47.843 UTC,2018-07-02 13:30:11.797 UTC,0.0,kimberly02,Fremont,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2739419,<p>POOL TABLES FOR SALE CONTEMPORARY CLASSIC C...,400.0,2013-09-02 10:00:27.57 UTC,2013-09-02 10:00:27.57 UTC,0.0,USA Made Pool Tables,California,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1440631,<p>I am a Software Engineer</p>\n,32.0,2012-06-06 19:21:00.903 UTC,2012-06-06 19:36:13.78 UTC,0.0,twinkle,United States,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
